<a href="https://colab.research.google.com/github/spendnetwork/charts/blob/master/number_of_buyers_deloitte_chart_opensourced.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import altair as alt

In [0]:
url = 'https://raw.githubusercontent.com/spendnetwork/charts/master/number_of_buyers_deloitte.csv'
df = pd.read_csv(url)
#print(df)

In [0]:
df['releasedate']=pd.to_datetime(df['releasedate']).dt.strftime('%m/%d/%Y')

In [0]:
df1=(df.groupby('releasedate').agg({'buyer': ['count']}))
#df1.head()

In [0]:
df1.columns = df1.columns.map('_'.join)
df1 = df1.reset_index()
#df1.head()

In [0]:
#df1['releasedate']=pd.to_datetime(df1['releasedate'].dt.strftime('%m/%d/%Y'))
#df1['releasedate']=pd.to_datetime(df1['releasedate']).dt.strftime('%m/%d/%Y') 
#df1.head()

In [0]:
  nearest = alt.selection(type='single', nearest=True, on='mouseover',fields=['releasedate'], empty='none')

In [0]:
        line=alt.Chart(df1, width=1200, height=250, title="Count of buyers of Deloitte LLP").mark_line(color='#91D739').encode(
            x=alt.X('yearmonth(releasedate):O',
                    axis=alt.Axis(tickCount=12, title="https://openopps.com", 
                grid=False, 
                labelFont='Helvetica', 
                labelFontSize=12, 
                labelAngle=0, 
                titleAlign='left',
                titleAnchor='start',
                titleFont='Helvetica',
                titleFontSize= 11,
                titleFontWeight= 300,
                titleX= -0,
                titleY= 30) 
                   ),
            y=alt.Y('sum(buyer_count):Q',
                     axis=alt.Axis(title="",
                                    format=".0s",)
                   ),
            )
        selectors = alt.Chart(df1).mark_point().encode(
            x='yearmonth(releasedate):O',
            opacity=alt.value(0),
        ).add_selection(
            nearest
        )

        points = line.mark_point().encode(
            opacity=alt.condition(nearest, alt.value(1), alt.value(0))
        )


        text = line.mark_text(align='left', dx=10, dy=-10).encode(
            text=alt.condition(nearest, 'sum(buyer_count):Q', alt.value(' '))
        )

        rules = alt.Chart(df1).mark_rule(color='gray').encode(
            x='yearmonth(releasedate):O',
        ).transform_filter(
            nearest
        )

        chart = alt.layer(
            line, selectors, points, rules, text
        ).configure_view(
          strokeWidth=0
        ).configure_title(fontSize=18, anchor='start',
                color= "#91D739", font='Helvetica', fontWeight=400)

        chart

alt.LayerChart(...)